In [1]:
%load_ext autoreload
%autoreload 2

# Importar entorno y familiarizarse

In [1]:
from boardgame2 import ReversiEnv
import numpy as np

# Crear 3 tipos de jugador
- Random: Selecciona uniformemente una de las acciones válidas
- Greedy: Selecciona la acción que le da más ganancia inmediata (cantidad de piezas que come). Si hay más de una acción que da máxima ganancia samplear uniformemente entre ellas
- Optimum (solo para 4x4): Usando resultados de la PI optima obtenida por policy iteration

Tener en cuenta que:
- ReversiEnv tiene los métodos get_valid y next_step y no es necesario mantener el estado del entorno
- env.PASS ([-1,  0]) es una acción valida posible y debería hacerse cuando no get_valid devuelve una matriz de ceros

Para el optimo en 4x4 bajar usar la PI obtenida en la notebook anterior guardado en /mdp

In [2]:
class GreedyPlayer():
    def __init__(self, player=1, board_shape=None, env=None, flatten_action=False):
        if (env is None) and (board_shape is None):
            print("board_shape and env can't be both None")
        if env is None:
            env = ReversiEnv(board_shape=board_shape)
        self.env = env
        self.player = player # player number. 1 o -1
        self.flatten_action = flatten_action
        self.board_shape = self.env.board.shape[0]
    
    def predict(self, board):
        # Implementar
        # Tiene que devoler la acción en la que come más piezas.
        # A igualdad de piezas comidas, samplear uniformemente
        valid_actions = np.argwhere(self.env.get_valid((board, self.player)) == 1)
        if len(valid_actions) == 0:
            print('pass')
            action = self.env.PASS
        else:
            moves_score = []
            for a in valid_actions:
                next_state, _, _, _ = self.env.next_step((board, self.player), a)
                moves_score.append(next_state[0].sum() * self.player)
            best_score = max(moves_score)
            best_actions = valid_actions[np.array(moves_score)==best_score]
            action = best_actions[np.random.randint(len(best_actions))]
        if self.flatten_action:
            return action[0] * self.board_shape + action[1]
        else:
            return action
        
class RandomPlayer():
    def __init__(self, player=1, board_shape=None, env=None, flatten_action=False):
        if (env is None) and (board_shape is None):
            print("board_shape and env can't be both None")
        if env is None:
            env = ReversiEnv(board_shape=board_shape)
        self.env = env
        self.player = player
        self.flatten_action = flatten_action
        self.board_shape = self.env.board.shape[0]
    
    def predict(self, board):
        # Muestrea aleatoriamente las acciones válidas
        # Puede usar la función creada en la notebook anterior
        valid_actions = np.argwhere(self.env.get_valid((board, self.player)) == 1)
        if len(valid_actions) == 0:
            print('pass')
            action = self.env.PASS
        else:
            action = valid_actions[np.random.randint(len(valid_actions))]
        if self.flatten_action:
            return action[0] * self.board_shape + action[1]
        else:
            return action
        

class DictPolicyPlayer():
    def __init__(self, player=1, board_shape=4, env=None, flatten_action=False, dict_folder='mdp/pi_mdp.npy'):
        self.pi_dict = np.load(dict_folder, allow_pickle=True).item()
        if env is None:
            env = ReversiEnv(board_shape=board_shape)
        self.player = player
        self.flatten_action = flatten_action
        self.board_shape = board_shape
    
    def predict(self, board):
        # Elegir la acción optima y devolverla
        board_tuple = tuple((board * self.player).reshape(-1))
        action = self.pi_dict[board_tuple]      
        if self.flatten_action:
            return action[0] * self.board_shape + action[1]
        else:
            return action
            # return [action // self.board_shape, action % self.board_shape]

In [3]:
gp = GreedyPlayer(player=1, board_shape=4)
rp = RandomPlayer(player=1, board_shape=4)

# Verificar que el pass funciona OK

In [4]:
board = np.array([
    [-1, 0, 0, 0],
    [0, 1, 1, 0],
    [0, 1, 1, 0],
    [0, 0, 0, 0]]
)
rp.predict(board)
gp.predict(board)

pass
pass


array([-1,  0])

# Completar la función que dado dos jugadores imprima estadísticas de las partidas

In [5]:
def arena_stats(Player_1, Player_2, board_shape, N=500):
    
    env = ReversiEnv(board_shape=board_shape)
    wins_as_first = 0
    wins_as_second = 0
    plays_as_first = 0
    plays_as_second = 0
    total_steps = 0
    player_1 = Player_1(player=1, board_shape=board_shape, flatten_action=False)
    player_2 = Player_2(player=1, board_shape=board_shape, flatten_action=False) # Implementar
    for i in range(N):
        # Aveces empieza un jugador, a veces el otro
        first_player = np.random.choice([-1, 1])
        player_1.player = first_player
        player_2.player = -first_player
        
        plays_as_first = plays_as_first + (first_player == 1)
        plays_as_second = plays_as_second + (first_player == -1)
        
        done = False
        n_steps = 0
        (board, player) = env.reset()
        
        while not done:
            if first_player == player:
                action = player_1.predict(board) # Juega el jugador 1
            else:
                action = player_2.predict(board) # Juega el jugador 2
            (board, player), reward, done, info = env.step(action)
            n_steps = n_steps + 1
        total_steps = total_steps + n_steps
        wins_as_first = wins_as_first + (reward == first_player) * (first_player == 1)
        wins_as_second = wins_as_second + (reward == first_player) * (first_player == -1)
    print(f'Wins as first: {wins_as_first/plays_as_first}')
    print(f'Wins as second: {wins_as_second/plays_as_second}')
    print(f'Plays as first: {plays_as_first}')
    print(f'Plays as second: {plays_as_second}')
    print(f'Avg game duration: {total_steps/N}')
        
    

In [6]:
arena_stats(DictPolicyPlayer, GreedyPlayer, 4, N=2000)

Wins as first: 0.8530318602261048
Wins as second: 1.0
Plays as first: 973
Plays as second: 1027
Avg game duration: 11.729


In [7]:
arena_stats(DictPolicyPlayer, RandomPlayer, 4, N=1000)

Wins as first: 0.8015564202334631
Wins as second: 1.0
Plays as first: 514
Plays as second: 486
Avg game duration: 11.677


In [8]:
arena_stats(RandomPlayer, DictPolicyPlayer, 4, N=1000)

Wins as first: 0.0
Wins as second: 0.15853658536585366
Plays as first: 508
Plays as second: 492
Avg game duration: 11.627


In [11]:
arena_stats(RandomPlayer, GreedyPlayer, 4, N=1000)

Wins as first: 1.0
Wins as second: 1.0
Plays as first: 486
Plays as second: 514
Avg game duration: 1.486


In [12]:
arena_stats(RandomPlayer, RandomPlayer, 4)

Wins as first: 1.0
Wins as second: 1.0
Plays as first: 238
Plays as second: 262
Avg game duration: 1.476


In [13]:
arena_stats(GreedyPlayer, GreedyPlayer, 4)

Wins as first: 1.0
Wins as second: 1.0
Plays as first: 262
Plays as second: 238
Avg game duration: 1.524


In [67]:
arena_stats(RandomPlayer, GreedyPlayer, 8, N=1000)

Wins as first: 0.41025641025641024
Wins as second: 0.359026369168357
Plays as first: 507
Plays as second: 493
Avg game duration: 57.811


# Guardar todas las clases de jugadores en un player.py para que luego se puedan importar de la siguiente forma:

from players import RandomPlayer

from players import GreedyPlayer